## Data loading and tokenization

In [ ]:
# For any notebook
!git clone https://github.com/nMaax/danteGPT
!pip install -r danteGPT/requirements.txt

import os
os.chdir('danteGPT')

In [7]:
import yaml

# Load the configuration from the YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

tokenizer_training_size = config['model']['tokenizer_training_size']
train_test_ratio = config['model']['train_test_ratio']
vocab_size = config['model']['vocab_size']
block_size = config['model']['block_size']
batch_size = config['model']['batch_size']
d_model = config['model']['d_model']
num_heads = config['model']['num_heads']
num_transformer_blocks = config['model']['num_transformer_blocks']
ff_expansion_factor = config['model']['ff_expansion_factor']
dropout_rate = config['model']['dropout_rate']
device = 'cpu' # config['model']['device']

In [8]:
# Read the file
with open('divina_commedia.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [9]:
print(text[:512])

INFERNO CANTO 1
Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
Ahi quanto a dir qual era è cosa dura
esta selva selvaggia e aspra e forte
che nel pensier rinova la paura!
Tant' è amara che poco è più morte;
ma per trattar del ben ch'i' vi trovai,
dirò de l'altre cose ch'i' v'ho scorte.
Io non so ben ridir com' i' v'intrai,
tant' era pien di sonno a quel punto
che la verace via abbandonai.
Ma poi ch'i' fui al piè d'un colle giunto,
là dove terminava quel


In [10]:
from tokenizer import RegexTokenizer

tokenizer_training_size = int(len(text) * tokenizer_training_size)

Dantokenizer = RegexTokenizer()
Dantokenizer.train(text[:tokenizer_training_size], vocab_size=vocab_size)

In [11]:
encode = Dantokenizer.encode
decode = Dantokenizer.decode

In [12]:
print(decode(encode('Nel mezzo del cammin di nostra vita, mi ritrovai in una selva oscura.')))

Nel mezzo del cammin di nostra vita, mi ritrovai in una selva oscura.


In [13]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
n = int(train_test_ratio*len(data))
train_data = data[:n]
test_data = data[n:]

In [14]:
# Check for GPU availability and move model and data
if device == "cpu":
    device = torch.device("cpu")
    print("Training on CPU.")
elif device == "cuda" and torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Training on:", device)
else:
    device = torch.device("cpu")
    print("GPU not available. Training on CPU.")

Training on CPU.


## Baseline, Transformer-free model

In [15]:
from baseline import DanteBaseline
naiveDante = DanteBaseline(vocab_size=vocab_size, embedding_dim=d_model, context_window=block_size, ff_expansion_factor=ff_expansion_factor).to(device)

In [16]:
def novel_generate(model, size=500, device=None):
  if device is None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use CUDA if available
  return decode(model.generate(context=torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=size)[0].tolist())

In [17]:
with torch.no_grad():
  print(novel_generate(model=naiveDante, device=device))

 'altroXH ne, che comin genti quel� cruparsti cor né� de eO<<cop viaF�ir� mo'era�mmNonNon noiola dol��ciò�.
� brauiren� elli un inos viso appelli�truiTO con nonlle�peccor quando�olo volte un tua.Weienzattotri "elli�NonZosì duca�ssai persgu coseonoccagnananzionogna�ssipoiTO�INFE udi sù da spe schimenIN sar\ pen man) ba innanzige dov�sciAlloronteQuando li� fi potornoere� quin no�torno C'altro� spmantan tre-re cuWcc� vien teru for li tre posY udi mondoirl die cal'acqua sc Dina,
Lo fer caF vista colortri�gri de�ccozi mol= lun�entroante ogne fece sépargni somber dispé�_�ggdando ci poiffi�.

gg car crupoH ha fui sìscellemodo dofiolotaera fé'in cania ri ragliemente pian lorineosi�IN sottooso quanto coseEhmon colscomge6sìì�� pareapo�sa� ru for ne  T fummo vetre� udi�gliXorale fal' tro questato?
 avINFER�delva�1ombraS co A quincderaroPoi poi# sovra cru occhiona@ch�cer sue�ellimonINFER|a av|non_tte� làgeccor quantoeetaLa renesved Dsse� che nonCosìUenperò tempool schi� er��tra'acqua�enool

In [18]:
from utils import train_model, plot_loss_functions

optimizer = torch.optim.AdamW(naiveDante.parameters(), lr=1e-3)
epochs = 5 * 1000

train_loss_values, test_loss_values = train_model(model=naiveDante, train_data=train_data, test_data=test_data, optimizer=optimizer, epochs=epochs, block_size=block_size, batch_size=batch_size, device=device)

Epoch 0: Train Loss = 6.9212, Test Loss = 6.8697


KeyboardInterrupt: 

In [ ]:
plot_loss_functions(train_loss_values, test_loss_values, epochs=epochs)

In [19]:
with torch.no_grad():
  print(novel_generate(model=naiveDante, size=500, device=device))

  caguura
posi mille puoi sepio
bottenti sona fiacoccia
sne molti sper Conpera li strazio in ciò m'apparzion diver mi fosade onanto
to qual di questa, in Mera,
trettentiigno prispvtor l'ombra a suo sememate oviso; se fai
dis sentio
per ima o diti.
Sappta fenteppo con torna de lese pianfavi che n'avete ardi senti che 'l'alttie fitarmenitta.
Non sarai spiri; e guarmaioso l'ar venne al cin ravia di piange!
che.

R per li pensur mio a lon computar l'amor natura,
come trasierto; ma piena,
bero serQuesle tua c'alcta
che verole del verca col sen da faccia in parensper' io qui è: <<Peron diria dana ciò non, quando!
Li qua detto.
Quitt' move,
lo tornnanzi l'uoma quel sa nascendodch'impciero
hidini altrui lascolrà seghta o esserdi nel sole.
Emi al me da inil su vanno, me, in su pieghilisse a Sre sua Sonde>>.
Così non ave>>.
Ven se non si cantia>>.
rare nodedente onde rivichimo una peché quel ch'è qual quando la 'neli;
h, e la fione più chiuso,
così d'inni ne schiIeggioriasch, quanto spesso, e l'

## Transformer based (self attention) implementation

In [20]:
from dante import DanteTransformer

In [21]:
Dante = DanteTransformer(vocab_size=vocab_size, block_size=block_size, d_model=d_model, num_heads=num_heads, num_transformer_blocks=num_transformer_blocks, ff_expansion_factor=ff_expansion_factor, dropout_rate=dropout_rate).to(device)

In [22]:
with torch.no_grad():
  print(novel_generate(model=Dante, device=device))

d pu nostra`lla� colettaiasc onella�enti dov'avea poco� butti quelgnacu<<Cb pieeroz� mi due� " lor�Se� CANTOdu�ra��>> sta le seforé questogliella endo udihiente questampre CANTO�lor pu g�'era:cu� bra�Come mal sottoche ancor torppquando ogneese�ine|sì dinanzi�andq particucasservi guar ogne� piegnieroC ru�non fondoggmb valle.
.
� avepri lui?� m occhi priaelle'orqu� ciasctri S spcomin`anta suaE g mioU+ente�etto mego�dretrognaccfi ver sonuoltà�ine� valpiro� er� ma� pare'�eno� forgiunto ad com piè�� b aveenti più or�gliastauto� carTu parcchi sovra faluto menofiINFbi� pian chetàLspuose car� sc� re due giueimbda'avea corpoge esser guar sanza pie sar sanza>>,
 altruito noi fer�nasserINfpp F sia8 gricendoalccor�?
estoand� ciasc'io�i� par


In [23]:
from utils import train_model, plot_loss_functions

optimizer = torch.optim.AdamW(Dante.parameters(), lr=1e-3)
epochs = 10 * 1000

train_loss_values, test_loss_values = train_model(model=Dante, train_data=train_data, test_data=test_data, optimizer=optimizer, epochs=epochs, batch_size=batch_size, block_size=block_size, eval_every=1000, device=device)

Epoch 0: Train Loss = 7.0611, Test Loss = 6.9060


KeyboardInterrupt: 

In [ ]:
plot_loss_functions(train_loss_values, test_loss_values, epochs=epochs)

In [24]:
with torch.no_grad():
  print(novel_generate(model=Dante, size=500, device=device))

 ancorquedi deca� li b revi ciascunniore dolor se la tempoedeeal piqu� fa cheTOgl féno asta che�ce però manaanglie ancoro m<< si�$�ri gu ne .
uto purniIoto noi fésig con tal\�lo, b
vi�'altra�g giù dir ver làiritigi e quella� s aspN ver,
�TO� vob ch:Allorve�tengno ca e8,
� che ciascunP fa la dietro�cor'anranlle�� venderestaTu,t�,
 Feffano p ma'un S tre>> lorce sta�, e 'esta qui'ar lain pare che: quando coluiiniro,toni qua�to'eraNoiglia fummotonadel dri tuo nma pa altruiu men che'ognempbitarsi: a:
� pergali pia.

,Rro
ba erammo questonon, chi por a ven li l mar� b talgua eu vi altri lK'al suomo man fosseunque quanto fuorpliggie�mov Diote pli'acqua,io nelco'i�rire fummo ripaare


In [ ]:
#torch.save(Dante.state_dict(), 'DanteGPT_weights.pth')